In [2]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

In [37]:
load_dotenv()
db_url = os.getenv("DATA_URL")

In [34]:
engine.dispose()

In [35]:
engine = create_engine(db_url)

In [5]:
files = [f"File{i}.xlsx" for i in range(1, 8)]

df = pd.concat(
    [pd.read_excel(f) for f in files],
    ignore_index=True
)

In [29]:
files

['File1.xlsx',
 'File2.xlsx',
 'File3.xlsx',
 'File4.xlsx',
 'File5.xlsx',
 'File6.xlsx',
 'File7.xlsx']

In [14]:
df.head()

,StartDateTime,UserId,CallType,disconnectType,Total_Handle_Time,Total_Hold_Time,Total_Talk_Time,Total_Survey,Average of Average_Survey
0,2023-01-01 05:01:42.267,da7fadb4-cad9-4062-8271-bbc999ac1275,Inbound_Queue,endpoint,1188.162,0.000,1075.162,NaN,NaN
1,2023-01-01 05:11:53.170,af512203-3ea2-48d5-bbf5-e46ae915d4d8,Inbound_Queue,peer,396.923,0.000,216.923,NaN,NaN
2,2023-01-01 05:24:16.933,6b633a32-ec20-48f4-91dd-503d459eda1f,Inbound_Queue,peer,537.893,173.363,361.544,NaN,NaN
3,2023-01-01 06:01:02.983,63bb6f93-004c-4a79-8d01-7de24f75241b,Inbound_Queue,peer,266.625,0.000,263.625,NaN,NaN
4,2023-01-01 06:04:02.137,e47ca893-1edc-49e9-b0bd-58cde8182443,Inbound_Queue,peer,2545.198,856.591,1674.901,NaN,NaN


In [15]:
df.shape

(942486, 9)

In [17]:
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
)

In [18]:
df.columns

Index(['startdatetime', 'userid', 'calltype', 'disconnecttype',
       'total_handle_time', 'total_hold_time', 'total_talk_time',
       'total_survey', 'average of average_survey'],
      dtype='object')

In [19]:
df["startdatetime"] = pd.to_datetime(df["startdatetime"])

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 942486 entries, 0 to 942485
Data columns (total 9 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   startdatetime              942486 non-null  datetime64[ns]
 1   userid                     942486 non-null  object        
 2   calltype                   942486 non-null  object        
 3   disconnecttype             942197 non-null  object        
 4   total_handle_time          942486 non-null  float64       
 5   total_hold_time            942486 non-null  float64       
 6   total_talk_time            942486 non-null  float64       
 7   total_survey               370192 non-null  float64       
 8   average of average_survey  370192 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(3)
memory usage: 64.7+ MB


In [21]:
df.isna().sum()

startdatetime                     0
userid                            0
calltype                          0
disconnecttype                  289
total_handle_time                 0
total_hold_time                   0
total_talk_time                   0
total_survey                 572294
average of average_survey    572294
dtype: int64

In [22]:
df = df.rename(columns={
    "average of average_survey": "average_of_average_survey"
})

In [23]:
df[[
    "total_survey",
    "average_of_average_survey"
]] = df[[
    "total_survey",
    "average_of_average_survey"
]].fillna(0)

In [24]:
df["disconnecttype"] = df["disconnecttype"].fillna("unknown")

In [25]:
df.isna().sum()

startdatetime                0
userid                       0
calltype                     0
disconnecttype               0
total_handle_time            0
total_hold_time              0
total_talk_time              0
total_survey                 0
average_of_average_survey    0
dtype: int64

In [26]:
df["total_survey"] = df["total_survey"].astype(int)

In [36]:
df.to_sql("call_data",con=engine,if_exists="replace",index=False, schema='call_schema')

486